In [ ]:
#kmeans with min and max size constriant

In [335]:
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)

def kmeans_plusplus_init(X, k):
    """Initialize the centroids using k-means++ algorithm."""
    centers = []
    first_center = X[np.random.choice(X.shape[0])]
    centers.append(first_center)

    for _ in range(1, k):
        distances = np.min(np.linalg.norm(X[:, np.newaxis] - np.array(centers), axis=2), axis=1)
        probabilities = distances / np.sum(distances)
        next_center = X[np.random.choice(X.shape[0], p=probabilities)]
        centers.append(next_center)

    return np.array(centers)

def reassign_points(X, labels, centers, k, min_size, max_size):
    """Reassign points to clusters based on the min_size and max_size constraints."""
    if X.shape[0] < k * min_size:
        raise ValueError(f"min_size is too large")
    if X.shape[0] > k * max_size:
        raise ValueError(f"max size is too small")

    changed = False
    distances = np.linalg.norm(X[:, np.newaxis] - centers, axis=2)

    for j in range(k):
        cluster_points = np.where(labels == j)[0]

        if len(cluster_points) < min_size:
            other_clusters = np.where(labels != j)[0]
            closest_points = np.argsort(distances[other_clusters, j])
            points_needed = min_size - len(cluster_points)
            labels[other_clusters[closest_points[:points_needed]]] = j
            changed = True

        elif len(cluster_points) > max_size:
            farthest_points = np.argsort(-distances[cluster_points, j])
            points_to_remove = len(cluster_points) - max_size
            for point_index in cluster_points[farthest_points[:points_to_remove]]:
                possible_clusters = np.delete(np.arange(k), j)
                labels[point_index] = possible_clusters[np.argmin(distances[point_index, possible_clusters])]
            changed = True

    return labels, changed

def kmeans_with_constraints(X, k, min_size, max_size, max_iter=100, tol=1e-4):
    """Perform K-means clustering with constraints on the cluster sizes."""
    if X.shape[0] < k:
        raise ValueError(f"Number of clusters k={k} cannot be greater than the number of data points {X.shape[0]}.")

    if min_size > max_size:
        raise ValueError("min_size cannot be greater than max_size.")

    centers = kmeans_plusplus_init(X, k)
    old_centers = np.zeros_like(centers)
    prev_labels = np.zeros(X.shape[0])
    convergence = False

    for i in range(max_iter):
        distances = np.linalg.norm(X[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        
        labels, changed_pre = reassign_points(X, labels, centers, k, min_size, max_size)
        
        new_centers = []
        for j in range(k):
            points_in_cluster = X[labels == j]
            if len(points_in_cluster) == 0:
                raise ValueError(f"Cluster {j} has zero points, try different initialization or constraints.")
            new_center = np.mean(points_in_cluster, axis=0)
            new_centers.append(new_center)

        centers = np.array(new_centers)

        # Check for convergence
        if np.all(np.abs(centers - old_centers) < tol):
            logging.info(f"Converged after {i} iterations.")
            convergence = True
            break

        old_centers = centers.copy()
        labels, changed_post = reassign_points(X, labels, centers, k, min_size, max_size)
        
        # Check if no further optimization within constraints is possible
        if not (changed_pre or changed_post):
            logging.info(f"Unable to further optimize within constraints after {i} iterations.")
            break

        prev_labels = labels.copy()

    if not convergence:
        logging.warning("Did not converge within max_iter. Consider adjusting parameters or increasing max_iter.")

    return centers, labels


